(Always be aware of your imports and preserve namespaces!!!)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

First, let's load the MNIST data and define some TF objects associated to it:

In [ ]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
nimg  = mnist.train.images.shape[0]
npix  = mnist.train.images.shape[1]
nside = int(np.sqrt(npix))
x     = tf.placeholder(tf.float32, shape=[None, npix])
y_    = tf.placeholder(tf.float32, shape=[None, 10])

---

### Models in Tensorflow, part 2: CNNs

The regression model is obviously way better than the 10 templates made of averages method (but bear in mind that we have a lot more parameters!).  But compared to state-of-the-art, 92% is not very good.  State-of-the-art methods use variants on convolutional neural nets (or CNNs or "conv nets").  Let's build one.

#### DEFINING THE MODEL

In the above example the $W$ and $b$ tensors were initialized to zeros.  This was OK in that example, but is not good practice for neural nets for several reasons, one of which being that all zeros may very well be a local extremum of the "loss surface".  I.e., the gradient may be zero and thus gradient descent won't go anywhere.  So let's define a function that initialize our weights and biases with some gaussian noise for the weights and a small positive value for the biases:

In [ ]:
def weight_variable(shape):
    """ Initialize a variable with Gaussian noise."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """ Initialize a variable with a constant value."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Why did we make functions to do this?  Because we are going to build a "deep" (but not too deep) neural network and so we are going to be creating several layers of these weights and biases.

The canonical structure of a conv net layer takes the form "convolve, activate, pool".  These layers are then stacked on top of each other some number of times.  If that number is at least two, it is considered a deep net.  So, since we're going to be doing this multiple times, let's define another couple of helper functions:

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Now we build the first layer.  First some slight modifications to the image tensor so that it can be fed into the net which takes advantage of the 2D structure of the image,

In [ ]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # last channel is color channel

Now we'll set the footprint of the convolutional patch to $5 \times 5$ and have it produce $32$ features,

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

We apply this convolution, the activation function, and the (2x2) pooling process:

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

So, what are the outputs of this layer going to be?  Well, for each input image, the output of conv2d in this layer will be $28 \times 28 \times 32$ (32 feature maps, each with the same number of pixels as the image).  Relu doesn't change the number of pixels since it is a pixel-wise activation.  But max_pool_2x2 reduces the size of each feature map by 2 in each dimension, so the final output of this layer for aeach image is $14 \times 14 \times 32$.

This get sent into the second layer that will again have the structure "convolve, activate, pool".  We'll keep the $5 \times 5$ footprint, but increase the number of output features to $64$.  As before, we'll keep the $2 \times 2$ pooling...

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

... and again apply this convolution, the activation, and the pooling process:

In [ ]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

The outputs of this layer are $14 \times 14 \times 64$ after convolution and relu, but then the $2 \times 2$ max pooling brings us to $7 \times 7 \times 64$.

We'll stop the convolutions here, but (as is common) tack on a fully connected layer to the end.  We'll give it $1024$ neurons.

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

But note that, just like we had to reshape the images to the correct dimensions for the convolutional layer to accept the inputs, we must do the same here with the output of the previous layer which (as described above) was $14 \times 14 \times 64$:

In [ ]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

We send this output into the fully connected layer (which will also use a relu activation function),

In [ ]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

We could stop here, and run this net as it is, but let's use our full bag of tricks by applying dropout.

In [ ]:
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

The dropout probability is set as a placeholder so that it is an input to the model and not hard-coded.

The last piece we need is a way to turn that fully connected layer into output probabilities for each integer for the input image just like in the regression case (the actual application of softmax will happen later).

In [ ]:
W_fc2  = weight_variable([1024, 10])
b_fc2  = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#### TRAINING THE MODEL

Now that we've built this model, it's time to train it!  We'll do this almost exactly like the regression case.  First we define a loss,

In [ ]:
# Previous loss:
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

and an optimizer,

In [ ]:
# Previous optimizer:
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Training will take quite a bit longer now, so let's keep an eye on how it's doing as it goes,

In [ ]:
# Previous prediction:
# correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

# Previous accuracy:
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(this time we won't use an interactive session)
Here we go:

In [ ]:
nstep = 200
loss  = np.zeros(nstep)

with tf.Session() as sess:
    # -- initialize **ALL** of those weights and biases
    sess.run(tf.global_variables_initializer())
    
    # -- loop through steps
    for ii in range(nstep):
        
        # get the next minibatch of images
        batch = mnist.train.next_batch(50)
        
        # alert the user how the training is going after every 10 steps
        if ii % 10 == 0:
            acc_in = {x:batch[0], y_:batch[1], keep_prob:1.0} # don't use dropout for accuracy estimate
            train_accuracy = accuracy.eval(feed_dict=acc_in)
            print("step {0}, training accuracy {1}".format(ii, train_accuracy))
            
        # take a gradient descent step
        mod_in = {x:batch[0], y_:batch[1], keep_prob:0.5}
        train_step.run(feed_dict=mod_in)
        loss[ii] = sess.run(cross_entropy, mod_in)
        
    # -- print the final accuracy on the test data
    test_in = {x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0} # don't use dropout for testing
    test_accuracy = accuracy.eval(feed_dict=test_in)
    print("test accuracy {0}".format(test_accuracy))

---